In [ ]:
import os
import numpy as np
import pandas as pd
import gc

In [ ]:
x_name = os.path.join(os.getcwd(), 'SCDC_Track1_2차', f"{'cst_feat_jan'}.csv")
df_x = pd.read_csv(x_name, index_col=0)

y_name = os.path.join(os.getcwd(), 'SCDC_Track1_2차', f"{'train'}.csv")
df_y = pd.read_csv(y_name, index_col=0)

v_name = os.path.join(os.getcwd(), 'SCDC_Track1_2차', f"{'val'}.csv")
df_v = pd.read_csv(v_name, index_col=0)

df_yv = pd.concat([df_y, df_v])
df = pd.merge(df_x, df_yv, on='cst_id_di')

In [ ]:
X = df.drop(columns = ['MRC_ID_DI'], axis=1)
y = df['MRC_ID_DI']

In [ ]:
X.info(memory_usage='deep')

In [ ]:
len(X.columns)*0.5

In [ ]:
import sklearn.feature_selection

select = sklearn.feature_selection.SelectKBest(k=113)
selected_features = select.fit(X, y)
indices_selected = selected_features.get_support(indices=True)
colnames_selected = [X.columns[i] for i in indices_selected]

X = X[colnames_selected]

In [ ]:
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

X = X.astype(np.float16)
combos = list(combinations(list(X.columns), 2))
colnames = list(X.columns) + ['_'.join(x) for x in combos]

poly = PolynomialFeatures(interaction_only=True, include_bias=False)
X = poly.fit_transform(X)
X = pd.DataFrame(X)
X.columns = colnames

gc.collect()

noint_indicies = [i for i, x in enumerate(list((X == 0).all())) if x]
X = X.drop(X.columns[noint_indicies], axis = 1)

In [ ]:
temp = X
temp['cst_id_di'] = y.index
temp = temp.set_index('cst_id_di')
temp['MRC_ID_DI'] = y

X = temp.drop(columns = ['MRC_ID_DI'], axis=1)
y = temp['MRC_ID_DI'].astype(np.int8)
gc.collect()

In [ ]:
import sklearn.feature_selection

select = sklearn.feature_selection.SelectKBest(k=331)
selected_features = select.fit(X, y)
indices_selected = selected_features.get_support(indices=True)
colnames_selected = [X.columns[i] for i in indices_selected]

X = X[colnames_selected]

gc.collect()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1, stratify = y)

In [ ]:
test = X_test
test['cst_id_di'] = y_test.index
test = test.set_index('cst_id_di')
test['MRC_ID_DI'] = y_test

X_test = test.drop(columns = ['MRC_ID_DI'], axis=1)
y_test = test['MRC_ID_DI']



train = X_train
train['cst_id_di'] = y_train.index
train = train.set_index('cst_id_di')
train['MRC_ID_DI'] = y_train

train_0 = train[train['MRC_ID_DI'] == 0].sample(frac=1)
train_1 = train[train['MRC_ID_DI'] == 1].sample(frac=1)
train_2 = train[train['MRC_ID_DI'] == 2].sample(frac=1)
train_3 = train[train['MRC_ID_DI'] == 3].sample(frac=1)
train_4 = train[train['MRC_ID_DI'] == 4].sample(frac=1)
train_5 = train[train['MRC_ID_DI'] == 5].sample(frac=1)
train_6 = train[train['MRC_ID_DI'] == 6].sample(frac=1)
train_7 = train[train['MRC_ID_DI'] == 7].sample(frac=1)
train_8 = train[train['MRC_ID_DI'] == 8].sample(frac=1)
train_9 = train[train['MRC_ID_DI'] == 9].sample(frac=1)
train_10 = train[train['MRC_ID_DI'] == 10].sample(frac=1)

sample_size = min(len(train_0),len(train_1), len(train_2), len(train_3), len(train_4), len(train_5), len(train_6), len(train_7),
                 len(train_8), len(train_9), len(train_10))

train_f = pd.concat([train_0.head(sample_size), train_1.head(sample_size), train_2.head(sample_size), train_3.head(sample_size),
                    train_4.head(sample_size), train_5.head(sample_size), train_6.head(sample_size), train_7.head(sample_size), 
                    train_8.head(sample_size), train_9.head(sample_size), train_10.head(sample_size)]).sample(frac=1)

X_train = train_f.drop(columns = ['MRC_ID_DI'], axis=1)
y_train = train_f['MRC_ID_DI']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1, stratify = y_train)

In [ ]:
"""
X_train[X_train.columns] = (X_train[X_train.columns]+1)/2*255
X_test[X_test.columns] = (X_test[X_test.columns]+1)/2*255
X_val[X_val.columns] = (X_val[X_val.columns]+1)/2*255
"""

In [ ]:
nth_img = 'image16'
img_dir = os.path.join(os.getcwd(), nth_img)
train_dir = os.path.join(os.getcwd(), nth_img, 'train')
test_dir = os.path.join(os.getcwd(), nth_img, 'test')
val_dir = os.path.join(os.getcwd(), nth_img, 'val')

if not os.path.exists(img_dir):
    os.makedirs(img_dir)
    
    train_dir = os.path.join(os.getcwd(), nth_img, 'train')
    test_dir = os.path.join(os.getcwd(), nth_img, 'test')
    val_dir = os.path.join(os.getcwd(), nth_img, 'val')
    os.makedirs(train_dir)
    os.makedirs(test_dir)
    os.makedirs(val_dir)
    
    for t in ['train', 'test', 'val']:
        for i in range(11):
            os.makedirs(os.path.join(os.getcwd(), nth_img, t, str(i)))

In [ ]:
repetitions = 331
p = 331

from scipy.special import softmax
np.set_printoptions(precision=5)

from PIL import Image
for ind, k in enumerate(y_train.index):
    print(ind)
    data = np.zeros((p, p, 3), dtype=np.float16)
    
    """
    an_array = X_train.iloc[ind].values
    row_array = np.tile(an_array, (repetitions, 1))
    col_array = np.transpose([an_array] * repetitions)
    final_array = (row_array + col_array) / 2
    final_array = np.repeat(final_array.flatten(), 3)
    data = final_array.reshape((p,p, 3))
    """
    
    """
    an_array = X_train.iloc[ind].values
    t_array = np.transpose([an_array])
    final_array = np.dot(np.resize(an_array, (p, 1)), np.resize(t_array,(1,p)))
    final_array = ((final_array + 1) /2) * 255 
    final_array = np.repeat(final_array.flatten(), 3)
    data = final_array.reshape((p,p, 3))
    """
    
    an_array = X_train.iloc[ind].values
    t_array = np.transpose([an_array])
    qk = np.dot(np.resize(an_array, (p, 1)), np.resize(t_array,(1,p)))
    qkv = np.dot(softmax( qk / np.sqrt(p), axis=1), qk)
    final_array = ((qkv + 1) /2) * 255 
    final_array = np.repeat(final_array.flatten(), 3)
    data = final_array.reshape((p,p, 3))
    
    
    img = Image.fromarray(data.astype(np.uint8), 'RGB')
    
    img_name = os.path.join(os.getcwd(), nth_img, 'train', str(y_train.iloc[ind]), str(k) + '.png')
    img.save(img_name)
    
    gc.collect()

In [ ]:
for ind, k in enumerate(y_test.index):
    print(ind)
    data = np.zeros((p, p, 3), dtype=np.float16)
    
    """
    an_array = X_test.iloc[ind].values
    row_array = np.tile(an_array, (repetitions, 1))
    col_array = np.transpose([an_array] * repetitions)
    final_array = (row_array + col_array) / 2
    final_array = np.repeat(final_array.flatten(), 3)
    data = final_array.reshape((p,p, 3))
    """
    
    """
    an_array = X_test.iloc[ind].values
    t_array = np.transpose([an_array])
    final_array = np.dot(np.resize(an_array, (p, 1)), np.resize(t_array,(1,p)))
    final_array = ((final_array + 1) /2) * 255 
    final_array = np.repeat(final_array.flatten(), 3)
    data = final_array.reshape((p,p, 3))
    """
    
    an_array = X_test.iloc[ind].values
    t_array = np.transpose([an_array])
    qk = np.dot(np.resize(an_array, (p, 1)), np.resize(t_array,(1,p)))
    qkv = np.dot(softmax( qk / np.sqrt(p), axis=1), qk)
    final_array = ((qkv + 1) /2) * 255 
    final_array = np.repeat(final_array.flatten(), 3)
    data = final_array.reshape((p,p, 3))
    
    img = Image.fromarray(data.astype(np.uint8), 'RGB')
    
    img_name = os.path.join(os.getcwd(), nth_img, 'test', str(y_test.iloc[ind]), str(k) + '.png')
    img.save(img_name)
    
    gc.collect()

In [ ]:
for ind, k in enumerate(y_val.index):
    print(ind)
    data = np.zeros((p, p, 3), dtype=np.float16)
    
    """
    an_array = X_val.iloc[ind].values
    row_array = np.tile(an_array, (repetitions, 1))
    col_array = np.transpose([an_array] * repetitions)
    final_array = (row_array + col_array) / 2
    final_array = np.repeat(final_array.flatten(), 3)
    data = final_array.reshape((p,p, 3))
    """
    
    """
    an_array = X_val.iloc[ind].values
    t_array = np.transpose([an_array])
    final_array = np.dot(np.resize(an_array, (p, 1)), np.resize(t_array,(1,p)))
    final_array = ((final_array + 1) /2) * 255 
    final_array = np.repeat(final_array.flatten(), 3)
    data = final_array.reshape((p,p, 3))
    """
    
    an_array = X_val.iloc[ind].values
    t_array = np.transpose([an_array])
    qk = np.dot(np.resize(an_array, (p, 1)), np.resize(t_array,(1,p)))
    qkv = np.dot(softmax( qk / np.sqrt(p), axis=1), qk)
    final_array = ((qkv + 1) /2) * 255 
    final_array = np.repeat(final_array.flatten(), 3)
    data = final_array.reshape((p,p, 3))
    
    img = Image.fromarray(data.astype(np.uint8), 'RGB')
    
    img_name = os.path.join(os.getcwd(), nth_img, 'val', str(y_val.iloc[ind]), str(k) + '.png')
    img.save(img_name)
    
    gc.collect()

In [ ]:
len(y_train), len(y_test), len(y_val)

In [ ]:
#(4742, 28501, 527)